In [ ]:

import heartpy as hp

hrdata = hp.get_data(r'D:\lemo下载\rr-interval-time-series-from-healthy-subjects-1.0.0\002.txt')
data, _ = hp.load_exampledata(0) #this example set is sampled at 100Hz

working_data, measures = hp.process(data, 128.0)
# print(working_data)
# print(measures['bpm']) #returns BPM value
# print(measures['rmssd']) # returns RMSSD HRV measure

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
